In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 200     # number of variables
num_ineq = 200    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_200-200.csv")

  6%|████▌                                                                           | 57/1000 [01:07<23:23,  1.49s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 12%|█████████▍                                                                     | 120/1000 [02:25<17:56,  1.22s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 17%|█████████████▏                                                                 | 167/1000 [03:24<14:52,  1.07s/it]

...

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


 70%|███████████████████████████████████████████████████████                        | 697/1000 [14:15<05:49,  1.15s/it]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [20:26<00:00,  1.23s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -46.730136         16.508989      0.332651
std       0.444228          2.188989      0.079426
min     -47.932033          9.702206      0.272455
25%     -47.025423         14.941754      0.293414
50%     -46.752522         16.485873      0.307633
75%     -46.441484         18.010361      0.346934
max     -45.188960         24.531775      1.151797
Number of infeasible solution: 1000


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 4627.95
Epoch 1, Validation Loss: -7.91
Epoch 2, Validation Loss: -17.02
Epoch 3, Validation Loss: -18.60
Epoch 4, Validation Loss: -20.66
Epoch 5, Validation Loss: -21.15
Epoch 6, Validation Loss: -22.71
Epoch 7, Validation Loss: -22.85
Epoch 8, Validation Loss: -24.34
Epoch 9, Validation Loss: -25.43
Epoch 10, Validation Loss: -25.74
Epoch 11, Validation Loss: -24.34
Epoch 12, Validation Loss: -27.45
Epoch 13, Validation Loss: -25.51
Epoch 14, Validation Loss: -27.36
Epoch 15, Validation Loss: -27.96
Epoch 16, Validation Loss: -28.34
Epoch 17, Validation Loss: -28.68
Epoch 18, Validation Loss: -27.77
Epoch 19, Validation Loss: -28.49
Epoch 20, Validation Loss: -28.00
Epoch 21, Validation Loss: -28.19
Epoch 22, Validation Loss: -29.17
Epoch 23, Validation Loss: -30.06
Epoch 24, Validation Loss: -28.31
Epoch 25, Validation Loss: -29.92
Epoch 26, Validation Loss: -30.27
Epoch 27, Validation Loss: -29.14
Epoch 28, Validation Loss: -27.57
Epoch 29, Validation Los

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_200-200.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:02<00:00,  8.17it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -32.294299          0.001352      0.004790
std       0.109061          0.013102      0.003123
min     -32.482080          0.000000      0.001617
25%     -32.405177          0.000000      0.003015
50%     -32.271355          0.000000      0.003608
75%     -32.210489          0.000000      0.005210
max     -32.101241          0.211506      0.022439
Number of infeasible solution: 22


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2502.10
Epoch 1, Validation Loss: -14.01
Epoch 2, Validation Loss: -19.96
Epoch 3, Validation Loss: -22.66
Epoch 4, Validation Loss: -25.59
Epoch 5, Validation Loss: -24.84
Epoch 6, Validation Loss: -24.03
Epoch 7, Validation Loss: -24.83
Epoch 8, Validation Loss: -25.53
Epoch 9, Validation Loss: -27.80
Epoch 10, Validation Loss: -25.46
Epoch 11, Validation Loss: -25.46
Epoch 12, Validation Loss: -28.18
Epoch 13, Validation Loss: -27.55
Epoch 14, Validation Loss: -29.21
Epoch 15, Validation Loss: -28.57
Epoch 16, Validation Loss: -28.18
Epoch 17, Validation Loss: -28.98
Epoch 18, Validation Loss: -27.26
Epoch 19, Validation Loss: -28.75
Epoch 20, Validation Loss: -28.69
Epoch 21, Validation Loss: -28.97
Epoch 22, Validation Loss: -29.57
Epoch 23, Validation Loss: -29.58
Epoch 24, Validation Loss: -30.53
Epoch 25, Validation Loss: -29.02
Epoch 26, Validation Loss: -27.01
Epoch 27, Validation Loss: -28.69
Epoch 28, Validation Loss: -29.69
Epoch 29, Validation Lo

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_200-200.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:03<00:00,  8.13it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -33.179023          0.016896      0.004696
std       0.415980          0.063015      0.003216
min     -33.788758          0.000000      0.001519
25%     -33.465495          0.000000      0.003024
50%     -33.260578          0.000000      0.003691
75%     -32.974211          0.000000      0.005041
max     -29.414661          0.648015      0.036511
Number of infeasible solution: 126
